# PassiveSSL

## Documentation

- Docs: https://www.circl.lu/services/passive-ssl/
- API Reference: https://github.com/adulau/crl-monitor/tree/master/client

## Exercises

#### PassiveSSL bootstrap


In [ ]:
import pypssl
import getpass

PASSIVE_SSL_USER = getpass.getpass("Enter your PassiveSSL User:")
PASSIVE_SSL_KEY = getpass.getpass("Enter your PassiveSSL API Password:")

pssl = pypssl.PyPSSL(basic_auth=(PASSIVE_SSL_USER, PASSIVE_SSL_KEY))

#### 1.0. Get certificates information of an IP
- *Hint*: related API endpoint: `https://www.circl.lu/v2pssl/query/[ip]`

In [ ]:
ip = '8.8.8.8'
data = pssl.query(ip)
print(f"\nPassiveSSL Data for {ip}\n{'='*40}")
print("Certificates found:")
for cert in data[ip]["certificates"]:
    print(f"  - {cert}")

print("\nSubjects:")
for cert_hash, subj_data in data[ip]["subjects"].items():
    print(f"  • {cert_hash}")
    for val in subj_data["values"]:
        print(f"      ↳ {val}")

#### 1.1. Get IPs seen with a given certificate
- *Hint*: related API endpoint: `https://www.circl.lu/pssl/cquery/[certificate_sha1]`

In [ ]:
cert_sha1 = 'c46fed822dadac3f31f9bb4d1a78a1d9eae4567b'
data = pssl.query_cert(cert_sha1)
print(data)

#### 1.2. Get X509 certificate
- *Hint*: related API endpoint: `https://www.circl.lu/pssl/cfetch/[certificate_sha1]`

In [ ]:
cert_sha1 = 'c46fed822dadac3f31f9bb4d1a78a1d9eae4567b'
data = pssl.fetch_cert(cert_sha1)
print(data["pem"])

#### 1.3. Identify expired certificates

In [ ]:
from datetime import datetime, UTC

def check_expired_certs(cert_infos):
    now = datetime.now(UTC)
    expired = []
    info = cert_info.get('info', {})
    
    # Collect data for timeline
    not_before = info.get('not_before')
    not_after = info.get('not_after')
    fingerprint = info.get('fingerprint', 'N/A')
    if not_after and not_after < now:
        expired.append((fingerprint, not_after))
    return expired

ip = '8.8.8.8'
data = pssl.query(ip)
for cert_sha1 in data[ip]['certificates']:
    try:
        cert_infos = pssl.fetch_cert(cert_sha1)
        expired_certs = check_expired_certs(cert_infos)
        print("Expired certificates:")
        for fp, exp_date in expired_certs:
            print(f"  Fingerprint: {fp} expired on {exp_date}")
    except Exception as ex:
        print(f"Error processing {cert_sha1}: {str(ex)}")



#### 1.4. List all unique issuers

In [ ]:
from collections import Counter

def issuer_counts(cert_infos):
    info = cert_infos.get('info', {})
    issuers = [info.get('issuer', 'Unknown') for cert in cert_infos]
    return Counter(issuers)

ip = '8.8.8.8'
data = pssl.query(ip)
for cert_sha1 in data[ip]['certificates']:
    try:
        cert_infos = pssl.fetch_cert(cert_sha1)
        issuers_count = issuer_counts(cert_infos)
        print("Certificate issuers count:")
        for issuer, count in issuers_count.items():
            print(f"  {issuer}: {count}")
    except Exception as ex:
        print(f"Error processing {cert_sha1}: {str(ex)}")


#### 1.5. Create timeline of the certificates validit period

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

ip = '8.8.8.8'
data = pssl.query(ip)

timeline_data = []

for cert_sha1 in data[ip]["certificates"]:
    try:
        cert_info = pssl.fetch_cert(cert_sha1)
        info = cert_info.get('info', {})
        
        # Collect data for timeline
        not_before = info.get('not_before')
        not_after = info.get('not_after')
        fingerprint = info.get('fingerprint', 'N/A')
        
        if not_before and not_after:
            timeline_data.append({
                'fingerprint': fingerprint,
                'not_before': not_before,
                'not_after': not_after
            })
    except Exception as ex:
        print(f"Error processing {cert_sha1}: {str(ex)}")

# Plotting the timeline
fig, ax = plt.subplots(figsize=(10, len(timeline_data)*0.6))

# Convert dates to matplotlib format
dates_before = [mdates.date2num(x['not_before']) for x in timeline_data]
dates_after = [mdates.date2num(x['not_after']) for x in timeline_data]
durations = [after - before for before, after in zip(dates_before, dates_after)]

y_pos = range(len(timeline_data))

ax.barh(y_pos, durations, left=dates_before, height=0.4, color='skyblue')

ax.set_yticks(y_pos)
ax.set_yticklabels([d['fingerprint'] for d in timeline_data])

ax.xaxis_date()
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

ax.set_xlabel('Certificate Validity Period')
ax.set_title(f'Certificate Timeline for IP: {ip}')
plt.tight_layout()
plt.show()
